# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier

nltk.download(['wordnet', 'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wiemb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wiemb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wiemb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster-Response.db')
df =  pd.read_sql_table("Disaster-Response" , con=engine)
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Defining feature and target variables
X=df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
     """
    Function: processing the message 
    Args:
      text(str): the message
    Return:
      lemmed: a list of the root form of the message words
    """
   # Normalize
     text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
   # Tokenize 
     words = word_tokenize(text)
   # Remove stop_words
     words = [w for w in words if w not in stopwords.words("english")]
   # Lemmatize
     lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
     return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Pipleine1 : grid Random Forest Classifier
pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
                   ])


In [6]:
pipeline1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000002245839DAE8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .30, random_state=42) 

In [8]:
X_train.shape

(18351,)

In [9]:
y_train.shape

(18351, 36)

In [10]:
y_test.shape

(7865, 36)

In [11]:
X_test.shape

(7865,)

In [12]:
#Train pipeline
pipeline1.fit(X_train , y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def Accuracy(y_test, y_pred):
    
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy)

In [14]:
y_pred = pipeline1.predict(X_test)
Accuracy(y_test, y_pred)

Accuracy: related                   0.811316
request                   0.898156
offer                     0.995423
aid_related               0.777241
medical_help              0.921805
medical_products          0.949905
search_and_rescue         0.976224
security                  0.981310
military                  0.970375
child_alone               1.000000
water                     0.956898
food                      0.940242
shelter                   0.936046
clothing                  0.985887
money                     0.979021
missing_people            0.988430
refugees                  0.967196
death                     0.958805
other_aid                 0.869421
infrastructure_related    0.935664
transport                 0.957152
buildings                 0.954609
electricity               0.979402
tools                     0.993897
hospitals                 0.990083
shops                     0.996440
aid_centers               0.986904
other_infrastructure      0.956771
weather_re

In [15]:
def report( X_test, y_test):
    '''
    Function to Report the f1 score, precision and recall for each output category of the dataset.
    Input:  test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [16]:
report( X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1873
           1       0.83      0.94      0.88      5934
           2       0.38      0.29      0.33        58

    accuracy                           0.81      7865
   macro avg       0.64      0.55      0.58      7865
weighted avg       0.80      0.81      0.79      7865

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.50      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.78      7865
weighted avg       0.89      0.90      0.89      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

C:\Users\wiemb\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7721
           1       0.00      0.00      0.00       144

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.96      0.98      0.97      7865

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7620
           1       0.73      0.08      0.14       245

    accuracy                           0.97      7865
   macro avg       0.85      0.54      0.56      7865
weighted avg       0.96      0.97      0.96      7865

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

water
              precision    recall  f1-score   

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
# Pipleine 2: grid search Classifier 
parameters =  {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv = GridSearchCV(pipeline1, param_grid=parameters,n_jobs=-1)


In [18]:
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [20]:
report( X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1873
           1       0.83      0.94      0.88      5934
           2       0.38      0.29      0.33        58

    accuracy                           0.81      7865
   macro avg       0.64      0.55      0.58      7865
weighted avg       0.80      0.81      0.79      7865

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.50      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.78      7865
weighted avg       0.89      0.90      0.89      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

C:\Users\wiemb\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.97      7160
           1       0.81      0.38      0.51       705

    accuracy                           0.94      7865
   macro avg       0.87      0.68      0.74      7865
weighted avg       0.93      0.94      0.93      7865

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7750
           1       0.70      0.06      0.11       115

    accuracy                           0.99      7865
   macro avg       0.84      0.53      0.55      7865
weighted avg       0.98      0.99      0.98      7865

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7695
           1       0.78      0.04      0.08       170

    accuracy                           0.98      7865
   macro avg       0.88      0.52      0.53      7865
weighted avg       0.97      0.98      0.97      7865

miss

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Pipleine 3: Adaboost Classifier 
pipeline3 = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [22]:
pipeline3.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000002245839DAE8>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMM

In [23]:
report( X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1873
           1       0.83      0.94      0.88      5934
           2       0.38      0.29      0.33        58

    accuracy                           0.81      7865
   macro avg       0.64      0.55      0.58      7865
weighted avg       0.80      0.81      0.79      7865

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.50      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.78      7865
weighted avg       0.89      0.90      0.89      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

C:\Users\wiemb\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.97      7160
           1       0.81      0.38      0.51       705

    accuracy                           0.94      7865
   macro avg       0.87      0.68      0.74      7865
weighted avg       0.93      0.94      0.93      7865

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7750
           1       0.70      0.06      0.11       115

    accuracy                           0.99      7865
   macro avg       0.84      0.53      0.55      7865
weighted avg       0.98      0.99      0.98      7865

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7695
           1       0.78      0.04      0.08       170

    accuracy                           0.98      7865
   macro avg       0.88      0.52      0.53      7865
weighted avg       0.97      0.98      0.97      7865

miss

### 9. Export your model as a pickle file

In [24]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.